# Initial Cleaning

In [1]:
#Install required libraries
!pip install pymongo
!pip install ssh-pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 13.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 21.9 MB/s eta 0:00:00
  Created wheel for ssh-pymongo: filename=ssh_pymongo-1.0.4-py3-none-any.whl size=2566 sha256=718e71520a3e929aae264c3aeaab50a5cd2989ac53af10e0c90370cb98db0172
  Stored in directory: /root/.cache/pip/wheels/1d/6a/9d/dee25ecdecdb43b31f2494250adceca24ac30f0f3993113be6
Successfully built ssh-pymongo


In [2]:
#import pymongo to connect to db
from pymongo import UpdateOne
from ssh_pymongo import MongoSession
import pprint

In [3]:
#Set host and remotely connect to mongodb
MONGO_HOST = "136.51.3.69"
MONGO_DB = "local"
MONGO_USER = "jeff"
MONGO_PASS = "password"
MONGO_PORT = 2222

session = MongoSession(MONGO_HOST,
    port=MONGO_PORT,
    user=MONGO_USER,
    password=MONGO_PASS,
    uri='mongodb:/136.51.3.69:2222'
)

Collections:

DIBBSAward2023-04-13

DIBBSAward2023-04-15

CICIAward2023-04-10

CSSIAward2023-04-16

MRIAward2023-04-16

CCFAward2023-04-11

SI2Award2023-04-24

In [4]:
#List of collection names to iterate over as stored in the db

#collection_names = ['DIBBSAward2023-04-13', 'DIBBSAward2023-04-15', 'CICIAward2023-04-10', 'CSSIAward2023-04-16']
collection_names = ['MRIAward2023-04-16', 'CCFAward2023-04-11', 'SI2Award2023-04-24']

In [5]:
#Set variables for each respective db
local = session.connection['local']
clean = session.connection['clean']

#Big regex string containing common words
nonowords = '(javascript)|(access denied)|(403.*error)|(403.*forbidden)|(can\'t find the page)|(404 not found)|(site under maintenance)|(down for maintenance)'

for curCollection in collection_names:
  print('Cleaning for ' + curCollection)
  print('-------------------')
  #set current collection to clean
  db = local[curCollection]
  print('Current # of documents: ' + str(db.count_documents({})))

  #get all NON empty documents and ones that DON'T match the regex
  getAllClean = list(db.find(
      {'$nor' : [{'raw html' : ''}, {'raw html' : { '$regex' : nonowords, '$options' : 'i'}}]}
  ))

  print('Documents after regex clean: ' + str(len(getAllClean)))

  #insert all found 'clean' documents into the clean db under the same collection name
  db_clean = clean[curCollection] 
  db_clean.insert_many(getAllClean)

  #delete documents that are too short (<400 chars)
  db_clean.delete_many({'$expr' : { '$lt': [{ '$strLenCP': '$raw html' }, 400]}})

  print('Documents after small truncation: ' + str(db_clean.count_documents({})))


Cleaning for MRIAward2023-04-16
-------------------
Current # of documents: 123115


KeyboardInterrupt: ignored

# Sentiment Analysis and Tokenization


In [ ]:
#Installs
import sys
!{sys.executable} -m pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.9 MB/s eta 0:00:00


In [ ]:
#Importing essential tools
import os
import re
import pandas as pd
import gensim
import gensim.corpora as corpora
import nltk
import pickle 
import pyLDAvis
import pyLDAvis.gensim
import seaborn as sns;

from textblob import TextBlob
from bson.objectid import ObjectId

from wordcloud import WordCloud
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

  and should_run_async(code)



In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'https'])

In [ ]:
def cleanString(text):
  text = re.sub('[,/\.!?\n|\-#]', '', text)
  text = re.sub(r' +', ' ', text)
  return text

def findSentiment(text):
  return TextBlob(text)
  
#Function removes punctuations from topics in long_string.
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#Function removes stop_words from long_string, returns list of strings in long_string.
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
clean = session.connection['clean']

for curCollection in collection_names:
  print('Currently cleaning: ' + curCollection)
  print('-------------------')
  db_clean = clean[curCollection]

  data = db_clean.find({})
  df = pd.DataFrame(list(data))

  print('Starting clean html...')
  df['clean html'] = df['raw html'].apply(cleanString)
  print('Done with clean html!\n')

  print('Starting sentiment...')
  df['sentiment'] = df['clean html'].apply(lambda x : TextBlob(x).polarity)
  print('Done with sentiment!\n')

  print('Starting subjectivity...')
  df['subjectivity'] = df['clean html'].apply(lambda x : TextBlob(x).subjectivity)
  print('Done with subjectivity!\n')

  print('Starting tokenization...')
  html_data = df['clean html'].tolist()
  html_words = list(sent_to_words(html_data))
  html_words = remove_stopwords(html_words)
  df['tokens'] = html_words
  print('Done with tokenization!\n')

  print('Writing all changes...\n')
  data_dict = df.to_dict('records')
  updates = []
  for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'tokens' : row.get('tokens')}}, upsert=False))
  db_clean.bulk_write(updates)

Currently cleaning: DIBBSAward2023-04-13
-------------------
Starting clean html...
Done with clean html!

Starting sentiment...
Done with sentiment!

Starting subjectivity...
Done with subjectivity!

Starting tokenization...
Done with tokenization!

Writing all changes...

Currently cleaning: DIBBSAward2023-04-15
-------------------
Starting clean html...
Done with clean html!

Starting sentiment...
Done with sentiment!

Starting subjectivity...
Done with subjectivity!

Starting tokenization...
Done with tokenization!

Writing all changes...

Currently cleaning: CICIAward2023-04-10
-------------------
Starting clean html...
Done with clean html!

Starting sentiment...
Done with sentiment!

Starting subjectivity...
Done with subjectivity!

Starting tokenization...
Done with tokenization!

Writing all changes...

Currently cleaning: CSSIAward2023-04-16
-------------------
Starting clean html...
Done with clean html!

Starting sentiment...
Done with sentiment!

Starting subjectivity...
D

In [ ]:
for curCollection in collection_names:
  db_clean = db_clean = clean[curCollection]
  db_clean.delete_many({'$expr' : { '$lt': [{ '$strLenCP': '$clean html' }, 300]}})

# Testing on single database

In [ ]:
data = db_clean.find({})

In [ ]:
df = pd.DataFrame(list(data))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       1621 non-null   object
 1   Award     1621 non-null   object
 2   keyword   1621 non-null   object
 3   title     1621 non-null   object
 4   text      1621 non-null   object
 5   raw html  1621 non-null   object
dtypes: object(6)
memory usage: 76.1+ KB


In [ ]:
def cleanString(text):
  text = re.sub('[,/\.!?\n|\-#]', '', text)
  text = re.sub(r' +', ' ', text)
  return text

In [ ]:
df['clean html'] = df['raw html'].apply(cleanString)

In [ ]:
df.head()

,_id,Award,keyword,title,text,raw html,clean html,tokens
0,6440840da8cdca1ec8c5e6f7,CSSI Award,agriculture,None,None,\n\n\n\n Cyberinfrastructure for Sustai...,Cyberinfrastructure for Sustained Scientific ...,"[ , cyberinfrastructure, sustained, scientific..."
1,6440840da8cdca1ec8c5e6fa,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\nCSSI\n\n\n\n\n\n\n\n\n\n\n\n\n...,CSSI0 Skip to Content CSSI News Awards Fellow ...,"[cssi0, skip, content, cssi, news, awards, fel..."
2,6440840da8cdca1ec8c5e6fc,CSSI Award,agriculture,None,None,\n\n\n\n\nCSSI Inc. Receives Major NextGen Eng...,CSSI Inc Receives Major NextGen Engineering Co...,"[cssi, inc, receives, major, nextgen, engineer..."
3,6440840da8cdca1ec8c5e6ff,CSSI Award,agriculture,None,None,\n\nFaculty & staff awards - Gustavson School ...,Faculty & staff awards Gustavson School of Bus...,"[faculty, &, staff, awards, gustavson, school,..."
4,6440840da8cdca1ec8c5e702,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\n\n\n\n\nSECAF\n\n\n\n\n\n\n\n\...,SECAFJoin NowLoginMy AccountSECAF ChatContact ...,"[secafjoin, nowloginmy, accountsecaf, chatcont..."


In [ ]:
def findSentiment(text):
  return TextBlob(text)

In [ ]:
df['sentiment'] = df['clean html'].apply(lambda x : TextBlob(x).polarity)
df['subjectivity'] = df['clean html'].apply(lambda x : TextBlob(x).subjectivity)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'https'])

In [ ]:
#Function removes punctuations from topics in long_string.
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

#Function removes stop_words from long_string, returns list of strings in long_string.
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [ ]:
html_data = df['clean html'].tolist()
html_words = list(sent_to_words(html_data))
html_words = remove_stopwords(html_words)

In [ ]:
df['tokens'] = html_words

In [ ]:
df.head()

,_id,Award,keyword,title,text,raw html,clean html,tokens
0,6440840da8cdca1ec8c5e6f7,CSSI Award,agriculture,None,None,\n\n\n\n Cyberinfrastructure for Sustai...,Cyberinfrastructure for Sustained Scientific ...,"[sustained, scientific, innovation, cssi, la, ..."
1,6440840da8cdca1ec8c5e6fa,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\nCSSI\n\n\n\n\n\n\n\n\n\n\n\n\n...,CSSI0 Skip to Content CSSI News Awards Fellow ...,"[cssi, skip, content, cssi, news, awards, fell..."
2,6440840da8cdca1ec8c5e6fc,CSSI Award,agriculture,None,None,\n\n\n\n\nCSSI Inc. Receives Major NextGen Eng...,CSSI Inc Receives Major NextGen Engineering Co...,"[cssi, inc, receives, major, nextgen, engineer..."
3,6440840da8cdca1ec8c5e6ff,CSSI Award,agriculture,None,None,\n\nFaculty & staff awards - Gustavson School ...,Faculty & staff awards Gustavson School of Bus...,"[faculty, staff, awards, gustavson, school, bu..."
4,6440840da8cdca1ec8c5e702,CSSI Award,agriculture,None,None,\n\n\n\n\n\n\n\n\n\n\n\nSECAF\n\n\n\n\n\n\n\n\...,SECAFJoin NowLoginMy AccountSECAF ChatContact ...,"[secafjoin, nowloginmy, accountsecaf, chatcont..."


In [ ]:
data_dict = df.to_dict('records')

In [ ]:
updates = []
for _, row in df.iterrows():
    updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'tokens' : row.get('tokens')}}, upsert=False))
    #updates.append( UpdateOne({'_id': row.get('_id')}, {'$set': {'sentiment': row.get('sentiment'), 'subjectivity': row.get('subjectivity'), 'clean html': row.get('clean html'), 'lemma tokens' : row.get('lemma_tokens')}}, upsert=False))

2023-04-21 05:35:50,103| ERROR   | Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active
ERROR:sshtunnel.SSHTunnelForwarder:Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [ ]:
db_clean.bulk_write(updates)

In [ ]:
id = "64407ef1a64b00c43c4e4ba2"
test = db_clean.find({'_id' : ObjectId(id)})

test_df = pd.DataFrame(list(test))
print(test_df)

2023-04-21 05:54:09,408| ERROR   | Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active
ERROR:sshtunnel.SSHTunnelForwarder:Could not establish connection from local ('127.0.0.1', 38673) to remote ('127.0.0.1', 27017) side of the tunnel: open new channel ssh error: SSH session not active


                        _id       Award     keyword title  text  \
0  64407ef1a64b00c43c4e4ba2  CSSI Award  assessment  None  None   

                                            raw html  \
0  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPre-Award Info...   

                                          clean html  sentiment  subjectivity  \
0  Pre-Award Information - Research and Sponsored...   0.033929      0.231199   

                                              tokens  
0  [pre-award, information, research, sponsored, ...  


In [ ]:
id = "64407ef1a64b00c43c4e4ba1"
test = db_clean.find({'_id' : ObjectId(id)})

test_df = pd.DataFrame(list(test))
print(test_df)

                        _id       Award     keyword title  text  \
0  64407ef1a64b00c43c4e4ba1  CSSI Award  assessment  None  None   

                                            raw html  
0  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...  


In [ ]:
session.stop()